In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [4]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1228,1.3903,0.1011,0.3652,-2.6994,-1.1768,-0.1044,0.2218,4.9165,0.8232,0.1442
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1239,1.3604,0.0656,0.3354,-2.5776,-1.0987,-0.1294,0.2287,4.1943,0.5297,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0       0.188888
 0.188888  1.0

In [8]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

scenarios = block_bootstrap(values(data), 10_000, block_size=60, random_block=true)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0605,0.0237
5,0.0615,0.0228
10,0.0619,0.0218
25,0.0614,0.0218


,Total_Stock_CPI,Price_10Y_CPI
1,0.1544,0.0866
5,0.0675,0.0417
10,0.0482,0.0307
25,0.0302,0.0202


,Total_Stock_CPI,Price_10Y_CPI
1,-0.8086,0.5055
5,-0.3036,0.2843
10,-0.3017,0.0922
25,-0.1901,0.0867


,Total_Stock_CPI,Price_10Y_CPI
1,1.0931,1.1022
5,-0.1043,0.8353
10,-0.2716,-0.1528
25,-0.1617,-0.1789


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.4396,-0.293,-0.2227,-0.0263,0.0825,0.1688,0.2755,0.3082,0.3446
5,-0.0938,-0.0697,-0.0516,0.0126,0.0708,0.1098,0.1616,0.1869,0.2071
10,-0.0508,-0.0353,-0.0236,0.0295,0.0659,0.0975,0.1348,0.1454,0.1575
25,-0.0119,-0.0005,0.0097,0.0413,0.0624,0.0826,0.1092,0.117,0.1259


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1635,-0.1266,-0.1007,-0.0339,0.0196,0.0766,0.1685,0.2267,0.2873
5,-0.0758,-0.061,-0.0428,-0.0035,0.0213,0.0488,0.0916,0.1117,0.1431
10,-0.0463,-0.0369,-0.027,-0.0,0.0214,0.0427,0.0729,0.0838,0.0941
25,-0.0215,-0.0156,-0.0108,0.0076,0.0216,0.0358,0.0548,0.0613,0.0692


In [7]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.376926  0.499561  0.516744  0.56925  0.70339

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 58.0  102.0  144.0  185.0  339.03

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.19893  0.376498  0.432131  0.496277  0.649711

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 84.0  161.0  235.0  348.0  688.03